In [ ]:
#  source :# https://www.thepythoncode.com/article/stock-price-prediction-in-python-using-tensorflow-2-and-keras

In [ ]:
!pip install tensorflow pandas numpy matplotlib yahoo_fin sklearn datetime requests config

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from yahoo_fin import stock_info as si
from collections import deque

import os
import numpy as np
import pandas as pd
import random

import requests, json
from config import *
from datetime import datetime

In [ ]:
API_KEY = ''
SECRETE_KEY=''
BASE_URL ='https://paper-api.alpaca.markets'
ACCOUNT_URL='{}/v2/account'.format(BASE_URL)
ORDERS_URL = '{}/v2/orders'.format(BASE_URL)
POSITION_URL = '{}/v2/position'.format(BASE_URL)
POSITIONS_URL= '{}/v2/positions'.format(BASE_URL)
ASSESTS_URL = '{}/v2/assets/QQQ'.format(BASE_URL)
HEADERS ={'APCA-API-KEY-ID':API_KEY ,  'APCA-API-SECRET-KEY':SECRETE_KEY}

In [ ]:
def get_final_df(model, data):
    
    
    buy_profit  = lambda current, pred_future, true_future: true_future - current if pred_future > current else 0
   
    sell_profit = lambda current, pred_future, true_future: current - true_future if pred_future < current else 0
    X_test = data["X_test"]
    y_test = data["y_test"]
    
    y_pred = model.predict(X_test)
    if SCALE:
        y_test = np.squeeze(data["column_scaler"]["close"].inverse_transform(np.expand_dims(y_test, axis=0)))
        y_pred = np.squeeze(data["column_scaler"]["close"].inverse_transform(y_pred))
    test_df = data["test_df"]
    
    test_df[f"close_{LOOKUP_STEP}"] = y_pred
    
    test_df[f"true_close_{LOOKUP_STEP}"] = y_test
    
    test_df.sort_index(inplace=True)
    final_df = test_df
    
   
    return final_df

In [ ]:
def shuffle_in_unison(a, b):
    
    state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(state)
    np.random.shuffle(b)

def load_data(ticker, n_steps=50, scale=True, shuffle=True, lookup_step=1, split_by_date=True,
                test_size=0.2, feature_columns= ['close', 'volume', 'open', 'high', 'low']):
    if isinstance(ticker, str):
       
        df_QQQ = si.get_data(ticker , start_date='01/01/1999'  )
        df_FTSE = si.get_data('^FTSE' , start_date='01/05/2000' )
        df_FTSE.drop(['adjclose','ticker'],1,inplace=True)
        df_QQQ.drop(['adjclose','ticker'],1,inplace=True)
        df_FTSE.rename({'open': 'FTSE_open', 'high': 'FTSE_high' , 'low': 'FTSE_low' , 'close':'FTSE_close' , 'volume':'FTSE_volume'}, axis=1, inplace=True)
       
        df = pd.concat([df_QQQ, df_FTSE], axis=1 ) 
        df= df_QQQ
    elif isinstance(ticker, pd.DataFrame):
       
        df = ticker
    else:
      print('error')     
   
    result = {}
    
    result['df'] = df.copy()
    
    for col in feature_columns:
        assert col in df.columns, f"'{col}' does not exist in the dataframe."

    if "date" not in df.columns:
        df["date"] = df.index
    if scale:
        column_scaler = {}
        
        for column in feature_columns:
            scaler = preprocessing.MinMaxScaler()
            df[column] = scaler.fit_transform(np.expand_dims(df[column].values, axis=1))
            column_scaler[column] = scaler
       
        result["column_scaler"] = column_scaler
   
    df['future'] = df['close'].shift(-lookup_step)
 
    last_sequence = np.array(df[feature_columns].tail(lookup_step))
    
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=n_steps)
    for entry, target in zip(df[feature_columns + ["date"]].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == n_steps:
            sequence_data.append([np.array(sequences), target])
  
    last_sequence = list([s[:len(feature_columns)] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)
 
    result['last_sequence'] = last_sequence
    
    X, y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        y.append(target)
 
    X = np.array(X)
    y = np.array(y)
    if split_by_date:
      
        train_samples = int((1 - test_size) * len(X))
        result["X_train"] = X[:train_samples]
        result["y_train"] = y[:train_samples]
        result["X_test"]  = X[train_samples:]
        result["y_test"]  = y[train_samples:]
        if shuffle:
            
            shuffle_in_unison(result["X_train"], result["y_train"])
            shuffle_in_unison(result["X_test"], result["y_test"])
    else:    
       
        result["X_train"], result["X_test"], result["y_train"], result["y_test"] = train_test_split(X, y, 
                                                                                test_size=test_size, shuffle=shuffle)
    
    dates = result["X_test"][:, -1, -1]
    
    result["test_df"] = result["df"].loc[dates]

    result["test_df"] = result["test_df"][~result["test_df"].index.duplicated(keep='first')]
    
    result["X_train"] = result["X_train"][:, :, :len(feature_columns)].astype(np.float32)
    result["X_test"] = result["X_test"][:, :, :len(feature_columns)].astype(np.float32)
    return result

In [ ]:
def create_model(sequence_length, n_features, units, cell, n_layers, dropout,
                loss, optimizer):
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
              model.add(cell(units, return_sequences=True, batch_input_shape=(None, sequence_length, n_features)))
        elif i == n_layers - 1:
            
              model.add(cell(units, return_sequences=False))
        else:
              model.add(cell(units, return_sequences=True))
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="relu"))
    model.compile(loss=loss, metrics=["mean_absolute_error"], optimizer=optimizer)
    return model

In [ ]:
def predict(model, data):

    last_sequence = data["last_sequence"][-N_STEPS:]

    last_sequence = np.expand_dims(last_sequence, axis=0)

    prediction = model.predict(last_sequence)
    
    if SCALE:
        predicted_price = data["column_scaler"]["close"].inverse_transform(prediction)[0][0]
    else:
        predicted_price = prediction[0][0]
    return predicted_price

In [ ]:
np.random.seed(314)
tf.random.set_seed(314)
random.seed(314)

In [ ]:
import os
import time
from tensorflow.keras.layers import LSTM

N_STEPS = 50

LOOKUP_STEP = 1

SCALE = True
scale_str = f"sc-{int(SCALE)}"

SHUFFLE = True
shuffle_str = f"sh-{int(SHUFFLE)}"

SPLIT_BY_DATE = False
split_by_date_str = f"sbd-{int(SPLIT_BY_DATE)}"

TEST_SIZE = 0.1

FEATURE_COLUMNS = ['close', 'volume', 'open', 'high', 'low' , 'FTSE_open' , 'FTSE_high' , 'FTSE_low', 'FTSE_close', 'FTSE_volume' ]

date_now = time.strftime("%Y-%m-%d")

N_LAYERS = 3

CELL = LSTM

UNITS = 256

DROPOUT = 0.3

BIDIRECTIONAL = False

LOSS = "huber_loss"
OPTIMIZER = tf.keras.optimizers.Adam(lr=0.0005 , decay= 1e-5)

BATCH_SIZE = 64
EPOCHS = 300

ticker = "QQQ"
ticker_data_filename = os.path.join("data", f"{ticker}_{date_now}.csv")

model_name = f"{date_now}_{ticker}-{shuffle_str}-{scale_str}-{split_by_date_str}-\
{LOSS}-{OPTIMIZER}-{CELL.__name__}-seq-{N_STEPS}-step-{LOOKUP_STEP}-layers-{N_LAYERS}-units-{UNITS}"
if BIDIRECTIONAL:
    model_name += "-b"


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
if not os.path.isdir("results"):
    os.mkdir("results")
if not os.path.isdir("logs"):
    os.mkdir("logs")
if not os.path.isdir("data"):
    os.mkdir("data")


data = load_data(ticker, N_STEPS, scale=SCALE, split_by_date=SPLIT_BY_DATE, 
                shuffle=SHUFFLE, lookup_step=LOOKUP_STEP, test_size=TEST_SIZE, 
                feature_columns=FEATURE_COLUMNS)

data["df"].to_csv(ticker_data_filename)

model = create_model(N_STEPS, len(FEATURE_COLUMNS), loss=LOSS, units=UNITS, cell=CELL, n_layers=N_LAYERS,
                    dropout=DROPOUT, optimizer=OPTIMIZER)

checkpointer = ModelCheckpoint(os.path.join("results", model_name + ".h5"), save_weights_only=True, save_best_only=True, verbose=1)
tensorboard = TensorBoard(log_dir=os.path.join("logs", model_name))

history = model.fit(data["X_train"], data["y_train"],
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(data["X_test"], data["y_test"]),
                    callbacks=[checkpointer, tensorboard],
                   
                    verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  from ipykernel import kernelapp as app


Epoch 1/300
81/81 [==============================] - ETA: 0s - loss: 0.0014 - mean_absolute_error: 0.0233
Epoch 1: val_loss improved from inf to 0.00017, saving model to results/2022-04-01_QQQ-sh-1-sc-1-sbd-0-huber_loss-<keras.optimizer_v2.adam.Adam object at 0x7f1440fcfc10>-LSTM-seq-50-step-1-layers-3-units-256.h5
81/81 [==============================] - 103s 1s/step - loss: 0.0014 - mean_absolute_error: 0.0233 - val_loss: 1.6821e-04 - val_mean_absolute_error: 0.0124
Epoch 2/300
81/81 [==============================] - ETA: 0s - loss: 3.3762e-04 - mean_absolute_error: 0.0151
Epoch 2: val_loss improved from 0.00017 to 0.00005, saving model to results/2022-04-01_QQQ-sh-1-sc-1-sbd-0-huber_loss-<keras.optimizer_v2.adam.Adam object at 0x7f1440fcfc10>-LSTM-seq-50-step-1-layers-3-units-256.h5
81/81 [==============================] - 82s 1s/step - loss: 3.3762e-04 - mean_absolute_error: 0.0151 - val_loss: 5.4673e-05 - val_mean_absolute_error: 0.0071
Epoch 3/300
81/81 [========================

In [ ]:
model_path = os.path.join("results", model_name) + ".h5"
model.load_weights(model_path)

In [ ]:

loss, mae = model.evaluate(data["X_test"], data["y_test"], verbose=0)

if SCALE:
    mean_absolute_error = data["column_scaler"]["close"].inverse_transform([[mae]])[0][0]
else:
    mean_absolute_error = mae

In [ ]:

final_df = get_final_df(model, data)

future_price = predict(model, data)

In [ ]:
print(f"Future price after {LOOKUP_STEP} days is {future_price:.2f}$")
print("Mean Absolute Error:", mean_absolute_error)

In [ ]:
#@title 預設標題文字
import matplotlib.pyplot as plt

def plot_graph(test_df):
    plt.plot(test_df[f'true_close_{LOOKUP_STEP}'], c='b')
    plt.plot(test_df[f'close_{LOOKUP_STEP}'], c='r')
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.legend(["Actual Price", "Predicted Price"])
    plt.show()


In [ ]:
plot_graph(final_df)

In [ ]:
def get_account():
  r = requests.get(ACCOUNT_URL , headers=HEADERS)
  return json.loads(r.content)

cash = float(get_account()['cash'])



In [ ]:
def create_order(symbol , qty , side , type , time_in_force):
  data= {"symbol": symbol , "qty": qty , "side": side , "type": type ,"time_in_force": time_in_force}
  r = requests.post(ORDERS_URL ,json=data, headers=HEADERS)
  return json.loads(r.content)

In [ ]:
def get_position(symbol ):
  data= {"symbol": symbol }
  r = requests.get(POSITIONS_URL,json=data, headers=HEADERS)
  return json.loads(r.content)

res = get_position(ticker)

shares=0
avg_price=0
lastday_price=0
if len(res) :
    print('ok' , res)
    shares = float(res[0]['qty'])
    avg_price =float(res[0]['avg_entry_price'])
    lastday_price = float(res[0]['lastday_price'])


else:
  print('no position')






In [ ]:
print(lastday_price)

In [ ]:
STRONG_BUY = 'strong buy'
BUY = 'buy'
NEUTRAL = 'neutral'
SELL= 'sell'
STRONG_SELL = 'strong sell'

In [ ]:
today = datetime.today().strftime('%d/%m/%Y')
today_price = si.get_data("QQQ" )['close'][-1]

In [ ]:
def signal(preditct_price , actual_price):
  difference =  ((preditct_price - actual_price) / (actual_price)) * 100
  print('Difference :' , difference)
  if difference >=2:
    return STRONG_BUY
  
  elif difference>=0.5 and difference < 2:
    return BUY

  elif difference< 0.5 and difference >-0.5:
    return NEUTRAL

  elif difference <= -0.5 and difference > -2:
    return SELL

  elif difference <= -2:
    return STRONG_SELL

  else:
    return None

In [ ]:
SIGNAL= signal(future_price, today_price)

In [ ]:
def trade(SIGNAL, cash , shares , lastday_price ):
 
  if SIGNAL == STRONG_BUY:
    cash *= 0.5
    total_share = int(cash/lastday_price)
    create_order(ticker , total_share , "buy", "market", "gtc")
    print( 'Singal :', SIGNAL )
    print('shares will buy ',total_share  )

  elif SIGNAL == BUY:
    cash *= 0.2
    total_share = int(cash/lastday_price)
    create_order(ticker, total_share , "buy", "market", "gtc")
    print( 'Singal :', SIGNAL )
    print('shares will buy',total_share )

  elif SIGNAL == NEUTRAL:
    print( 'Singal :', SIGNAL )


  elif SIGNAL == SELL:
    shares *= 0.5
    shares = int(shares)
    create_order(ticker, shares , "sell", "market", "gtc")

  elif SIGNAL == STRONG_SELL:
    shares = int(shares)
    create_order(ticker, shares , "sell", "market", "gtc")
  
  else:
    print('NO actions')


trade(SIGNAL, cash , shares , today_price)
print('Signal : ', SIGNAL)
print('Shares are holding', shares)
print('Cash in hand', cash)
print('Average position' , avg_price)
print('Predict price' , future_price)
print('today_price', today_price)

In [ ]:

trade(SIGNAL, cash , shares , today_price)
print('Signal : ', SIGNAL)
print('Shares are holding', shares)
print('Cash in hand', cash)
print('Average position' , avg_price)
print('Predict price' , future_price)
print('today_price', today_price)